# JupyterHub与Kubernetes GPU集成系统

## 项目概述

本notebook实现了一个完整的系统，将JupyterHub项目集成到主项目中，支持将Python脚本转换为Kubernetes GPU Job并提交到K8s集群。

### 主要功能
- 🔗 JupyterHub项目集成和配置
- 🎯 GPU资源查询和智能调度
- 📦 Python脚本自动化容器化
- 🚀 K8s Job自动提交和监控
- 💾 NFS存储集成和结果管理
- 📊 完整的任务监控和日志系统

### 技术栈
- **Python**: Kubernetes客户端、JupyterHub API
- **Kubernetes**: GPU节点调度、Job管理
- **NFS**: 分布式存储解决方案
- **Docker**: 容器化运行环境

## 1. 环境设置和依赖安装

安装必要的Python库，配置基础环境变量和常量定义。

In [ ]:
# 安装必要的依赖包
import subprocess
import sys

def install_package(package):
    """安装Python包"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} 安装成功")
    except subprocess.CalledProcessError:
        print(f"❌ {package} 安装失败")

# 安装主要依赖
required_packages = [
    "kubernetes>=24.2.0",
    "pyyaml>=6.0",
    "requests>=2.28.0",
    "aiohttp>=3.8.0",
    "jinja2>=3.1.0",
    "psutil>=5.9.0",
    "docker>=6.0.0"
]

for package in required_packages:
    install_package(package)

In [ ]:
# 导入所需的库
import os
import json
import yaml
import time
import logging
import asyncio
from datetime import datetime, timedelta
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Any
from dataclasses import dataclass

# Kubernetes 相关导入
from kubernetes import client, config
from kubernetes.client.rest import ApiException

# 配置日志
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler('jupyterhub-k8s-integration.log')
    ]
)
logger = logging.getLogger(__name__)

# 环境变量和常量配置
@dataclass
class Config:
    """系统配置类"""
    # K8s集群配置
    K8S_NAMESPACE: str = "jupyterhub-gpu"
    K8S_CONFIG_PATH: str = os.getenv("KUBECONFIG", "~/.kube/config")
    
    # NFS存储配置
    NFS_SERVER: str = os.getenv("NFS_SERVER", "nfs-server.default.svc.cluster.local")
    NFS_PATH: str = os.getenv("NFS_PATH", "/shared/jupyterhub")
    NFS_MOUNT_PATH: str = "/shared"
    
    # JupyterHub配置
    JUPYTERHUB_API_URL: str = os.getenv("JUPYTERHUB_API_URL", "http://localhost:8000/hub/api")
    JUPYTERHUB_API_TOKEN: str = os.getenv("JUPYTERHUB_API_TOKEN", "")
    
    # GPU配置
    GPU_VENDOR_NVIDIA: str = "nvidia.com/gpu"
    GPU_VENDOR_AMD: str = "amd.com/gpu"
    GPU_MEMORY_LIMIT: str = "8Gi"
    
    # 作业配置
    JOB_IMAGE_BASE: str = "jupyter/scipy-notebook:latest"
    JOB_TIMEOUT_SECONDS: int = 3600  # 1小时超时
    JOB_RETRY_LIMIT: int = 3
    
    # 第三方项目路径
    THIRD_PARTY_PATH: str = "../third-party"
    JUPYTERHUB_PATH: str = f"{THIRD_PARTY_PATH}/jupyterhub"

# 全局配置实例
config_instance = Config()

print("✅ 环境配置初始化完成")
print(f"📁 JupyterHub路径: {config_instance.JUPYTERHUB_PATH}")
print(f"🌐 K8s命名空间: {config_instance.K8S_NAMESPACE}")
print(f"💾 NFS服务器: {config_instance.NFS_SERVER}")
print(f"🎯 GPU内存限制: {config_instance.GPU_MEMORY_LIMIT}")

## 2. JupyterHub项目集成配置

读取third_party下的JupyterHub项目配置，解析项目结构，建立与主项目的集成接口。

In [ ]:
class JupyterHubIntegrator:
    """JupyterHub集成管理器"""
    
    def __init__(self, config: Config):
        self.config = config
        self.jupyterhub_path = Path(config.JUPYTERHUB_PATH)
        self.scripts_registry = {}
        
    def scan_jupyterhub_project(self) -> Dict[str, Any]:
        """扫描JupyterHub项目结构"""
        project_info = {
            "path": str(self.jupyterhub_path),
            "exists": self.jupyterhub_path.exists(),
            "config_files": [],
            "example_notebooks": [],
            "python_modules": [],
            "spawner_configs": []
        }
        
        if not project_info["exists"]:
            logger.warning(f"JupyterHub项目路径不存在: {self.jupyterhub_path}")
            return project_info
            
        # 扫描配置文件
        for pattern in ["*.py", "*.yaml", "*.yml", "*.json"]:
            project_info["config_files"].extend(
                list(self.jupyterhub_path.glob(pattern))
            )
            
        # 扫描示例notebooks
        examples_path = self.jupyterhub_path / "examples"
        if examples_path.exists():
            project_info["example_notebooks"] = list(examples_path.glob("**/*.ipynb"))
            
        # 扫描Python模块
        jupyterhub_module = self.jupyterhub_path / "jupyterhub"
        if jupyterhub_module.exists():
            project_info["python_modules"] = list(jupyterhub_module.glob("**/*.py"))
            
        # 查找spawner配置
        spawner_files = list(self.jupyterhub_path.glob("**/spawner*.py"))
        project_info["spawner_configs"] = spawner_files
        
        logger.info(f"✅ 扫描到 {len(project_info['config_files'])} 个配置文件")
        logger.info(f"✅ 扫描到 {len(project_info['example_notebooks'])} 个示例notebook")
        logger.info(f"✅ 扫描到 {len(project_info['python_modules'])} 个Python模块")
        
        return project_info
    
    def extract_notebook_scripts(self, notebook_path: Path) -> List[Dict[str, Any]]:
        """从notebook中提取Python脚本"""
        scripts = []
        
        try:
            with open(notebook_path, 'r', encoding='utf-8') as f:
                notebook_content = json.load(f)
                
            for i, cell in enumerate(notebook_content.get('cells', [])):
                if cell.get('cell_type') == 'code':
                    source = ''.join(cell.get('source', []))
                    if source.strip():
                        scripts.append({
                            "cell_index": i,
                            "source": source,
                            "notebook": str(notebook_path),
                            "has_gpu_code": self._detect_gpu_usage(source),
                            "estimated_resources": self._estimate_resources(source)
                        })
                        
        except Exception as e:
            logger.error(f"解析notebook失败 {notebook_path}: {e}")
            
        return scripts
    
    def _detect_gpu_usage(self, code: str) -> bool:
        """检测代码是否使用GPU"""
        gpu_indicators = [
            "cuda", "gpu", "torch.cuda", "tf.config.experimental",
            "tensorflow-gpu", "cupy", "numba.cuda", ".to('cuda')",
            "device='cuda'", "nvidia-smi"
        ]
        
        return any(indicator in code.lower() for indicator in gpu_indicators)
    
    def _estimate_resources(self, code: str) -> Dict[str, str]:
        """估算代码资源需求"""
        resources = {
            "cpu": "1",
            "memory": "2Gi",
            "gpu": "0"
        }
        
        # 基于代码特征估算资源需求
        if "torch" in code or "tensorflow" in code:
            resources["memory"] = "4Gi"
            
        if self._detect_gpu_usage(code):
            resources["gpu"] = "1"
            resources["memory"] = "8Gi"
            
        if "multiprocessing" in code or "concurrent" in code:
            resources["cpu"] = "2"
            
        # 检测大数据处理
        if any(word in code for word in ["pandas", "numpy", "large", "big"]):
            resources["memory"] = "8Gi"
            
        return resources

# 初始化JupyterHub集成器
integrator = JupyterHubIntegrator(config_instance)

# 扫描JupyterHub项目
project_info = integrator.scan_jupyterhub_project()

print("📊 JupyterHub项目扫描结果:")
for key, value in project_info.items():
    if isinstance(value, list):
        print(f"  {key}: {len(value)} 项")
    else:
        print(f"  {key}: {value}")

# 如果存在示例notebooks，解析第一个作为演示
if project_info["example_notebooks"]:
    first_notebook = project_info["example_notebooks"][0]
    scripts = integrator.extract_notebook_scripts(first_notebook)
    print(f"\n🔍 解析示例notebook: {first_notebook.name}")
    print(f"  提取到 {len(scripts)} 个代码片段")
    
    for i, script in enumerate(scripts[:3]):  # 只显示前3个
        print(f"  片段 {i+1}: GPU需求={script['has_gpu_code']}, 资源={script['estimated_resources']}")

## 3. Kubernetes客户端初始化

初始化Kubernetes Python客户端，配置集群连接参数，验证API访问权限。

In [ ]:
class KubernetesManager:
    """Kubernetes集群管理器"""
    
    def __init__(self, config: Config):
        self.config = config
        self.v1_core = None
        self.v1_batch = None
        self.v1_apps = None
        self.custom_api = None
        self.cluster_info = {}
        
    def initialize_client(self) -> bool:
        """初始化Kubernetes客户端"""
        try:
            # 尝试加载集群内配置
            try:
                config.load_incluster_config()
                logger.info("✅ 使用集群内配置连接K8s")
            except:
                # fallback到本地配置文件
                config.load_kube_config(config_file=self.config.K8S_CONFIG_PATH)
                logger.info("✅ 使用本地配置文件连接K8s")
            
            # 初始化API客户端
            self.v1_core = client.CoreV1Api()
            self.v1_batch = client.BatchV1Api()
            self.v1_apps = client.AppsV1Api()
            self.custom_api = client.CustomObjectsApi()
            
            return True
            
        except Exception as e:
            logger.error(f"❌ K8s客户端初始化失败: {e}")
            return False
    
    def verify_cluster_access(self) -> Dict[str, Any]:
        """验证集群访问权限并获取集群信息"""
        cluster_status = {
            "connected": False,
            "namespace_exists": False,
            "permissions": {},
            "cluster_version": "",
            "node_count": 0,
            "gpu_nodes": 0
        }
        
        try:
            # 检查API服务器连接
            version = self.v1_core.get_api_resources()
            cluster_status["connected"] = True
            logger.info("✅ K8s API服务器连接成功")
            
            # 获取集群版本
            version_info = self.v1_core.get_api_versions()
            cluster_status["cluster_version"] = str(version_info.versions)
            
            # 检查命名空间
            try:
                self.v1_core.read_namespace(name=self.config.K8S_NAMESPACE)
                cluster_status["namespace_exists"] = True
                logger.info(f"✅ 命名空间 {self.config.K8S_NAMESPACE} 存在")
            except ApiException as e:
                if e.status == 404:
                    logger.warning(f"⚠️ 命名空间 {self.config.K8S_NAMESPACE} 不存在，将创建")
                    self._create_namespace()
                    cluster_status["namespace_exists"] = True
                    
            # 获取节点信息
            nodes = self.v1_core.list_node()
            cluster_status["node_count"] = len(nodes.items)
            
            # 统计GPU节点
            gpu_nodes = 0
            for node in nodes.items:
                if self._node_has_gpu(node):
                    gpu_nodes += 1
                    
            cluster_status["gpu_nodes"] = gpu_nodes
            
            # 检查权限
            cluster_status["permissions"] = self._check_permissions()
            
            self.cluster_info = cluster_status
            logger.info(f"📊 集群信息: {cluster_status['node_count']} 节点, {cluster_status['gpu_nodes']} GPU节点")
            
        except Exception as e:
            logger.error(f"❌ 集群访问验证失败: {e}")
            
        return cluster_status
    
    def _create_namespace(self):
        """创建命名空间"""
        try:
            namespace = client.V1Namespace(
                metadata=client.V1ObjectMeta(
                    name=self.config.K8S_NAMESPACE,
                    labels={
                        "name": self.config.K8S_NAMESPACE,
                        "purpose": "jupyterhub-gpu-jobs"
                    }
                )
            )
            self.v1_core.create_namespace(body=namespace)
            logger.info(f"✅ 创建命名空间: {self.config.K8S_NAMESPACE}")
        except Exception as e:
            logger.error(f"❌ 创建命名空间失败: {e}")
    
    def _node_has_gpu(self, node) -> bool:
        """检查节点是否有GPU"""
        allocatable = node.status.allocatable or {}
        capacity = node.status.capacity or {}
        
        gpu_resources = [
            self.config.GPU_VENDOR_NVIDIA,
            self.config.GPU_VENDOR_AMD,
            "nvidia.com/gpu",
            "amd.com/gpu"
        ]
        
        for gpu_resource in gpu_resources:
            if gpu_resource in allocatable or gpu_resource in capacity:
                return True
                
        return False
    
    def _check_permissions(self) -> Dict[str, bool]:
        """检查必要的K8s权限"""
        permissions = {
            "list_nodes": False,
            "create_jobs": False,
            "list_jobs": False,
            "delete_jobs": False,
            "create_pods": False,
            "list_pods": False
        }
        
        try:
            # 检查节点列表权限
            self.v1_core.list_node(limit=1)
            permissions["list_nodes"] = True
            
            # 检查Job相关权限
            self.v1_batch.list_namespaced_job(namespace=self.config.K8S_NAMESPACE, limit=1)
            permissions["list_jobs"] = True
            permissions["create_jobs"] = True  # 如果能list通常也能create
            
            # 检查Pod相关权限
            self.v1_core.list_namespaced_pod(namespace=self.config.K8S_NAMESPACE, limit=1)
            permissions["list_pods"] = True
            permissions["create_pods"] = True
            
        except Exception as e:
            logger.warning(f"权限检查时出现错误: {e}")
            
        return permissions

# 初始化Kubernetes管理器
k8s_manager = KubernetesManager(config_instance)

# 初始化客户端并验证连接
print("🔗 正在初始化Kubernetes连接...")
if k8s_manager.initialize_client():
    print("✅ Kubernetes客户端初始化成功")
    
    # 验证集群访问
    print("🔍 正在验证集群访问权限...")
    cluster_status = k8s_manager.verify_cluster_access()
    
    print("\n📊 集群状态:")
    for key, value in cluster_status.items():
        print(f"  {key}: {value}")
        
    if cluster_status["connected"] and cluster_status["gpu_nodes"] > 0:
        print(f"\n🎉 集群连接成功！发现 {cluster_status['gpu_nodes']} 个GPU节点")
    elif cluster_status["connected"]:
        print("\n⚠️ 集群连接成功，但未发现GPU节点")
    else:
        print("\n❌ 集群连接失败")
else:
    print("❌ Kubernetes客户端初始化失败")

## 4. GPU资源查询和节点标签检测

实现查询K8s集群GPU资源的函数，检测节点标签和污点，获取GPU型号和可用性信息。

In [ ]:
@dataclass
class GPUNodeInfo:
    """GPU节点信息"""
    name: str
    gpu_count: int
    gpu_type: str
    gpu_memory: str
    available_gpus: int
    labels: Dict[str, str]
    taints: List[Dict[str, str]]
    schedulable: bool

@dataclass 
class GPUResourceStatus:
    """GPU资源状态"""
    total_gpus: int
    available_gpus: int
    used_gpus: int
    gpu_nodes: List[GPUNodeInfo]
    
class GPUResourceManager:
    """GPU资源管理器"""
    
    def __init__(self, k8s_manager: KubernetesManager):
        self.k8s_manager = k8s_manager
        self.gpu_cache = {}
        self.cache_timestamp = None
        self.cache_ttl = 60  # 缓存60秒
        
    def get_gpu_resource_status(self, use_cache: bool = True) -> GPUResourceStatus:
        """获取集群GPU资源状态"""
        current_time = time.time()
        
        # 检查缓存
        if (use_cache and self.cache_timestamp and 
            current_time - self.cache_timestamp < self.cache_ttl):
            logger.info("📋 使用缓存的GPU资源信息")
            return self.gpu_cache.get('status')
            
        logger.info("🔍 正在查询集群GPU资源状态...")
        
        try:
            nodes = self.k8s_manager.v1_core.list_node()
            gpu_nodes = []
            total_gpus = 0
            available_gpus = 0
            
            for node in nodes.items:
                gpu_info = self._analyze_gpu_node(node)
                if gpu_info and gpu_info.gpu_count > 0:
                    gpu_nodes.append(gpu_info)
                    total_gpus += gpu_info.gpu_count
                    available_gpus += gpu_info.available_gpus
                    
            status = GPUResourceStatus(
                total_gpus=total_gpus,
                available_gpus=available_gpus,
                used_gpus=total_gpus - available_gpus,
                gpu_nodes=gpu_nodes
            )
            
            # 更新缓存
            self.gpu_cache['status'] = status
            self.cache_timestamp = current_time
            
            logger.info(f"✅ GPU资源统计: 总计{total_gpus}卡, 可用{available_gpus}卡, 已用{total_gpus-available_gpus}卡")
            
            return status
            
        except Exception as e:
            logger.error(f"❌ 查询GPU资源失败: {e}")
            return GPUResourceStatus(0, 0, 0, [])
    
    def _analyze_gpu_node(self, node) -> Optional[GPUNodeInfo]:
        """分析单个节点的GPU信息"""
        node_name = node.metadata.name
        labels = node.metadata.labels or {}
        
        # 检查节点是否可调度
        schedulable = not (node.spec.unschedulable or False)
        
        # 解析污点
        taints = []
        if node.spec.taints:
            for taint in node.spec.taints:
                taints.append({
                    "key": taint.key,
                    "value": taint.value or "",
                    "effect": taint.effect
                })
        
        # 检查GPU资源
        capacity = node.status.capacity or {}
        allocatable = node.status.allocatable or {}
        
        gpu_count = 0
        gpu_type = "unknown"
        gpu_memory = "unknown"
        
        # 检查NVIDIA GPU
        if self.k8s_manager.config.GPU_VENDOR_NVIDIA in capacity:
            gpu_count = int(capacity[self.k8s_manager.config.GPU_VENDOR_NVIDIA])
            gpu_type = self._extract_gpu_type_from_labels(labels, "nvidia")
            
        # 检查AMD GPU
        elif self.k8s_manager.config.GPU_VENDOR_AMD in capacity:
            gpu_count = int(capacity[self.k8s_manager.config.GPU_VENDOR_AMD])
            gpu_type = self._extract_gpu_type_from_labels(labels, "amd")
        
        if gpu_count == 0:
            return None
            
        # 计算可用GPU数量
        available_gpus = self._calculate_available_gpus(node_name, gpu_count)
        
        return GPUNodeInfo(
            name=node_name,
            gpu_count=gpu_count,
            gpu_type=gpu_type,
            gpu_memory=gpu_memory,
            available_gpus=available_gpus,
            labels=labels,
            taints=taints,
            schedulable=schedulable
        )
    
    def _extract_gpu_type_from_labels(self, labels: Dict[str, str], vendor: str) -> str:
        """从节点标签提取GPU型号"""
        gpu_type_keys = [
            f"{vendor}.com/gpu-type",
            f"{vendor}.com/gpu.product",
            "accelerator",
            "gpu-type",
            "gpu.type"
        ]
        
        for key in gpu_type_keys:
            if key in labels:
                return labels[key]
                
        # 通用GPU型号检测
        for key, value in labels.items():
            if "gpu" in key.lower() and any(model in value.lower() 
                for model in ["rtx", "gtx", "tesla", "a100", "v100", "t4"]):
                return value
                
        return f"{vendor}-gpu"
    
    def _calculate_available_gpus(self, node_name: str, total_gpus: int) -> int:
        """计算节点可用GPU数量"""
        try:
            # 查询节点上的Pod资源使用情况
            pods = self.k8s_manager.v1_core.list_pod_for_all_namespaces(
                field_selector=f"spec.nodeName={node_name}"
            )
            
            used_gpus = 0
            for pod in pods.items:
                if pod.status.phase in ["Running", "Pending"]:
                    for container in pod.spec.containers:
                        if container.resources and container.resources.requests:
                            gpu_request = container.resources.requests.get(
                                self.k8s_manager.config.GPU_VENDOR_NVIDIA, "0"
                            )
                            if gpu_request != "0":
                                used_gpus += int(gpu_request)
                                
            return max(0, total_gpus - used_gpus)
            
        except Exception as e:
            logger.warning(f"计算节点 {node_name} 可用GPU时出错: {e}")
            return total_gpus  # 保守估计，返回总数
    
    def find_suitable_gpu_nodes(self, required_gpus: int = 1, 
                               gpu_type_preference: str = None) -> List[GPUNodeInfo]:
        """查找适合的GPU节点"""
        status = self.get_gpu_resource_status()
        suitable_nodes = []
        
        for node in status.gpu_nodes:
            # 检查基本条件
            if (node.schedulable and 
                node.available_gpus >= required_gpus):
                
                # 检查GPU类型偏好
                if gpu_type_preference:
                    if gpu_type_preference.lower() in node.gpu_type.lower():
                        suitable_nodes.append(node)
                else:
                    suitable_nodes.append(node)
        
        # 按可用GPU数量降序排序
        suitable_nodes.sort(key=lambda x: x.available_gpus, reverse=True)
        
        logger.info(f"🎯 找到 {len(suitable_nodes)} 个适合的GPU节点")
        return suitable_nodes

# 初始化GPU资源管理器
gpu_manager = GPUResourceManager(k8s_manager)

# 查询GPU资源状态
print("🔍 正在查询集群GPU资源...")
gpu_status = gpu_manager.get_gpu_resource_status(use_cache=False)

print(f"\n📊 GPU资源概览:")
print(f"  总GPU数量: {gpu_status.total_gpus}")
print(f"  可用GPU: {gpu_status.available_gpus}")
print(f"  已用GPU: {gpu_status.used_gpus}")
print(f"  GPU节点数: {len(gpu_status.gpu_nodes)}")

print(f"\n🖥️ GPU节点详情:")
for i, node in enumerate(gpu_status.gpu_nodes):
    print(f"  节点 {i+1}: {node.name}")
    print(f"    GPU类型: {node.gpu_type}")
    print(f"    GPU数量: {node.gpu_count} (可用: {node.available_gpus})")
    print(f"    可调度: {node.schedulable}")
    print(f"    污点数量: {len(node.taints)}")
    
    # 显示重要标签
    important_labels = {k: v for k, v in node.labels.items() 
                       if any(keyword in k.lower() for keyword in ['gpu', 'accelerator', 'instance-type'])}
    if important_labels:
        print(f"    重要标签: {important_labels}")
    print()

# 测试查找适合的GPU节点
if gpu_status.available_gpus > 0:
    print("🎯 测试查找适合的GPU节点:")
    suitable_nodes = gpu_manager.find_suitable_gpu_nodes(required_gpus=1)
    for node in suitable_nodes[:3]:  # 显示前3个最佳节点
        print(f"  推荐节点: {node.name} (可用GPU: {node.available_gpus})")
else:
    print("⚠️ 当前没有可用的GPU资源")

## 5. Python脚本解析和预处理

解析JupyterHub中的Python脚本，提取资源需求，识别GPU依赖，进行脚本验证和预处理。